## Find PACE and EMIT granules overlapping in time and space
Adapted from VITALS "Finding Concurrent ECOSTRESS and EMIT Data" tutorial located [here](https://nasa.github.io/VITALS/python/01_Finding_Concurrent_Data.html).

Adapted for PACE by: Skye Caplan (NASA, SSAI)


<div class="alert alert-info" role="alert">

An [Earthdata Login][edl] account is required to access data from the NASA Earthdata system, including NASA PACE and EMIT data.

</div>

[edl]: https://urs.earthdata.nasa.gov/
[oci-data-access]: https://oceancolor.gsfc.nasa.gov/resources/docs/tutorials/notebooks/oci_data_access/
[emit-data-access]: https://nasa.github.io/VITALS/python/Exploring_EMIT_L2A_RFL.html

## Summary

This notebook will use `earthacces` and metadata in EMIT and PACE files' attributes to find overlapping granules both in space and within a certain time interval. 

## Learning Objectives

At the end of this notebook, you will know how to:
- Search for both PACE and EMIT data
- Filter those search results to find concurrent datasets based on overpass time, cloud cover, etc.
- Download only the datasets which match your filter criteria. 

## Contents
1. [Setup](#1.-Setup)
2. [Filtering Data](#2.-Filtering-Data)
3. [Download or Stream Data](#3.-Download-or-Stream-Data)


## 1. Setup

Begin by importing all of the packages used in this notebook. PACE OCI has many data products.

In [ ]:
import utils
import os
import folium
import earthaccess
import warnings
import folium.plugins
import pandas as pd
import geopandas as gpd
import numpy as np 

from branca.element import Figure
from IPython.display import display
from shapely import geometry
from skimage import io
from datetime import timedelta
from shapely.geometry.polygon import orient
from matplotlib import pyplot as plt

The following cells use `earthaccess` to configure and persist Earthdata login credentials, then establish the parameters used for the data search. Specifically, we set the time range, define the region of interest (ROI), and specify the target products. In this example, we searched for concurrent reflectance data from both EMIT and PACE sensors.
Note that there are two PACE product suites listed in the `prods` list, [`PACE_OCI_L2_SFREFL`](https://doi.org/10.5067/PACE/OCI/L2/SFREFL/3.1) and `PACE_OCI_L2_SFREFL_NRT`. `NRT` in `PACE_OCI_L2_SFREFL_NRT` stands for Near Real Time, which is the collection including the most recent data (~1-2 months from present day). `PACE_OCI_L2_SFREFL` is for Refined data, which has been reprocessed with the best available ancillary data. More information can be found in the [SFREFL ATBD](https://www.earthdata.nasa.gov/apt/documents/sfrefl/v1.0).

In [ ]:
auth = earthaccess.login(persist=True)

In [ ]:
# Set up search params for datasets

# Temporal range
tspan = ('2024-12-01','2025-02-01')
# Bounding box is adjusted to the format earthaccess requires
roi_bounds = (128, -30, 134, -23) #wsen
bbox = geometry.box(*roi_bounds, ccw=True)
roi = list(bbox.exterior.coords)

gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[bbox])

# Data collections to search
prods = ["PACE_OCI_L2_SFREFL", "PACE_OCI_L2_SFREFL_NRT", "EMITL2ARFL"]

In [ ]:
# Grab results from Earthdata
results = earthaccess.search_data(
    short_name=prods,
    polygon=roi,
    temporal=tspan,
    count=500
)

len(results)

Now we can examine the results of our search, The result returned approximately 200 granules for this case. Because not all of these granules overlap spatially or temporally, we further filter the results to retain only those that overlap with each other.

## 2. Filtering Data

The returned results are converted to a dataframe and converted to a GeoDataframe. Next, only the colummns needed are kept and renamed. 

In [ ]:
# Create Dataframe of Results Metadata
results_df = pd.json_normalize(results)
# Create shapely polygons for result
geometries = [utils.get_shapely_object(results[index]) for index in results_df.index.to_list()]
# Convert to GeoDataframe
gdf = gpd.GeoDataFrame(results_df, geometry=geometries, crs="EPSG:4326")
# Remove results df, no longer needed
# del results_df
# Add browse imagery links
gdf['browse'] = [utils.get_png(granule) for granule in results]
gdf['shortname'] = [result['umm']['CollectionReference']['ShortName'] for result in results]
# Preview GeoDataframe
print(f'{gdf.shape[0]} granules total')

In [ ]:
# Create a list of columns to keep
keep_cols = ['meta.concept-id','meta.native-id', 'umm.TemporalExtent.RangeDateTime.BeginningDateTime',
             'umm.TemporalExtent.RangeDateTime.EndingDateTime','umm.CloudCover','umm.DataGranule.DayNightFlag',
             'geometry','browse', 'shortname']
# Remove unneeded columns
gdf = gdf[gdf.columns.intersection(keep_cols)]
# Rename some Columns
gdf.rename(columns = {'meta.concept-id':'concept_id','meta.native-id':'granule',
                       'umm.TemporalExtent.RangeDateTime.BeginningDateTime':'start_datetime',
                      'umm.TemporalExtent.RangeDateTime.EndingDateTime':'end_datetime',
                      'umm.CloudCover':'cloud_cover',
                      'umm.DataGranule.DayNightFlag':'day_night'}, inplace=True)
gdf['datetime_obj'] = pd.to_datetime(gdf['start_datetime'], format='ISO8601')

In [ ]:
gdf.head()

The dataframe is separated into two dataframes, one including the EMIT granules and the other one for PACE granules.

In [ ]:
# Suppress Setting with Copy Warning - not applicable in this use case
pd.options.mode.chained_assignment = None  # default='warn'

# Split into two dataframes - ECO and EMIT
pace_gdf = gdf[gdf['granule'].str.contains('OCI')]
emit_gdf = gdf[gdf['granule'].str.contains('EMIT')]
print(f' PACE OCI Granules: {pace_gdf.shape[0]} \n EMIT Granules: {emit_gdf.shape[0]}')

The dataframe is updated to only keep the rows with an existing intersection with EMIT granules. A new Colum

In [ ]:
## Create new column based on intersection with union of EMIT polygons.
pace_gdf['intersects'] = pace_gdf.intersects(emit_gdf.union_all())
## Apply subsetting
pace_gdf = pace_gdf[pace_gdf['intersects'] == True]
print(f' PACE OCI Granules: {pace_gdf.shape[0]} \n EMIT Granules: {emit_gdf.shape[0]}')

In [ ]:
pace_gdf, emit_gdf = utils.concurrent_match(pace_gdf,emit_gdf, col_name='datetime_obj',time_delta=timedelta(minutes=60))
print(f' PACE OCI Granules: {pace_gdf.shape[0]} \n EMIT Granules: {emit_gdf.shape[0]}')

And now we can plot our very reduced list of granules to see what we're left with. We want to make sure the PACE and EMIT granules we select have optimal conditions within themselves (e.g., clouds, no bad data, etc.), but also between sensors as well. EMIT granules are much smaller than one PACE OCI scene, and it's possible some of the EMIT data in our filtered list could overlap with PACE OCI at the edge of its swath, where L2 OCI pixels are expected to have greater angular effects. To minimize these effects, it's preferable to choose EMIT granules closer to the centre of an OCI scene, which the Folium map below is very useful for determining. 

In [ ]:
# Plot Using Folium
# Function to convert a bounding box for use in leaflet notation

# Create Figure and Select Background Tiles
fig = Figure(width="750px", height="375px")
map1 = folium.Map(tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', attr='Google')
fig.add_child(map1)

# Plot STAC ECOSTRESS Results - note we must drop the datetime_obj columns for this to work
pace_gdf.drop(columns=['datetime_obj']).explore(
    "granule",
    categorical=True,
    tooltip=[
        "granule",
        "start_datetime",
        "cloud_cover",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="PACE OCI",
    m=map1,
    legend=False
)

# Plot STAC EMITL2ARFL Results - note we must drop the datetime_obj columns for this to work
emit_gdf.drop(columns=['datetime_obj']).explore(
    "granule",
    categorical=True,
    tooltip=[
        "granule",
        "start_datetime",
        "cloud_cover",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="EMIT",
    m=map1,
    legend=False
)


folium.GeoJson(bbox,
                name='bounding_box',
                ).add_to(map1)

map1.fit_bounds(bounds=utils.convert_bounds(gdf.union_all().bounds))
map1.add_child(folium.LayerControl())
display(fig)

As mentioned above, we also want to make sure the data isn't filled with NaNs. From the map, it looks like January 17th is a good day for both EMIT and PACE, and the EMIT granules are relatively close to centre swath. We can print out the browse imagery for that day and decide which EMIT granules to keep for our final stream/download. 

PACE OCI browse imagery is coming soon, and once available, we will be able to quickly visualize the imagery in the same way to ensure initial data quality. 

In [ ]:
# filter by granule
pace_0117 = pace_gdf[(pace_gdf['granule'].str.contains("20250117T044314")) & ~(pace_gdf['granule'].str.contains("NRT"))]
emit_0117 = emit_gdf[emit_gdf['granule'].str.contains("20250117")]

cols = 3
rows = int(np.ceil(len(emit_0117)/cols))
fig, ax = plt.subplots(rows, cols, figsize=(20,20))
ax = ax.flatten()

for _n, index in enumerate(emit_0117.index.to_list()):
    img = io.imread(emit_0117['browse'][index])
    ax[_n].imshow(img)
    ax[_n].set_title(f"Index: {index} - {emit_0117['granule'][index]}")
    ax[_n].axis('off')
plt.tight_layout()
plt.show()


We'll keep 5 granules from the above selection, and put them in a final `filtered_results` list

In [ ]:
# Filter out granules to keep
good_grans = [78,80,81,87,90] # Index numbers
emit_0117_grans = emit_0117[emit_0117.index.isin(good_grans)]
keep_granules = pace_0117.index.to_list()+emit_0117_grans.index.to_list()
keep_granules.sort()

filtered_results = [result for i, result in enumerate(results) if i in keep_granules]
filtered_results

## 3. Download or Stream Data

Now that we have all the links for the data we want, we can either stream or download the files, depending on user preference. we can download all the files using `earthaccess.download(filtered_results, local_path="data")` or further filter the files to only downlaod the files we need. 

In [ ]:
# Download with earthaccess
emit_files = [filtered_results[1],filtered_results[2]]
pace_files = [filtered_results[-1]]


In [ ]:
# only get the EMIT RFL file and exclude the mask and uncertainty
emit_rfl = [result.data_links()[0] for result in emit_files]

In [ ]:
earthaccess.download(pace_files, local_path="data")



In [ ]:
earthaccess.download(emit_rfl, local_path="data")

Now the datasets are downloaded! Please proceed to the next tutorial in this series to learn how to open, explore, and regrid the scenes. 